#Exploratory Data Analysis on the landplots

# 0. Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from collections import Counter
from google.colab import drive

In [2]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
feature_frame=pd.read_parquet('gdrive/MyDrive/Colab Notebooks/Zrive/muestreos_parcelas.parquet')
feature_frame

,generated_muestreos,codparcela,provincia,municipio,fecha,campaña,poligono,parcela,recinto,subrecinto,estado_fenologico_1,estado_fenologico_2,estado_fenologico_3,estado_fenologico_4,estado_fenologico_5,estado_fenologico_6,estado_fenologico_7,estado_fenologico_8,estado_fenologico_9,estado_fenologico_10,estado_fenologico_11,estado_fenologico_12,estado_fenologico_13,estado_fenologico_14,porcentaje_floracion,generated_parcelas,102_coordenada_x_(utm),103_coordenada_y_(utm),104_altitud_(m),105_comarca,201_superf_cultivada_en_la_parcela_agrícola_(ha),202_pendiente_(%),203_orientación,204_textura_del_suelo,206_secano_/_regadío,211_utilización_de_cubierta_vegetal,212_tipo_de_cubierta_vegetal,214_cultivo_asociado/otro_aprovechamiento,301_marco_(m_x_m),302_densidad_(plantas/ha),303_nº_pies_por_árbol,304_formación,305_diámetro_de_copa_(m),308_variedad_principal,311_fecha_de_plantación_variedad_principal,208_riego:_procedencia_del_agua,209_riego:_calidad_del_agua,313_variedad_secundaria,107_zona_homogénea,120_zona_biológica_raif,401_estación_climática_asociada,402_sensor_climático_asociado,207_riego:_sistema_usual_de_riego,108_u_h_c_a_la_que_pertenece,316_fecha_de_plantación_variedad_secundaria,315_patrón_variedad_secundaria,317_%_superficie_ocupada_variedad_secundaria,306_altura_de_copa_(m),310_patrón_variedad_principal,411_representa_a_la_u_h_c_(si/no),109_sistema_para_el_cumplimiento_gestión_integrada
0,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-04-16,2006,23,109,0,0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-23 14:19:57,520481.06,4111107.12,863.0,RIO NACIMIENTO,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,NaN,NaN,NaN,LECHIN,NaN,NaN,NaN,NaN,AL/OL/02,AL/OL/02 NACIMIENTO - ALTO ANDARAX,AL006,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-04-27,2006,23,109,0,0,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-23 14:19:57,520481.06,4111107.12,863.0,RIO NACIMIENTO,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,NaN,NaN,NaN,LECHIN,NaN,NaN,NaN,NaN,AL/OL/02,AL/OL/02 NACIMIENTO - ALTO ANDARAX,AL006,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-05-04,2006,23,109,0,0,NaN,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-23 14:19:57,520481.06,4111107.12,863.0,RIO NACIMIENTO,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,NaN,NaN,NaN,LECHIN,NaN,NaN,NaN,NaN,AL/OL/02,AL/OL/02 NACIMIENTO - ALTO ANDARAX,AL006,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-05-11,2006,23,109,0,0,NaN,NaN,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-23 14:19:57,520481.06,4111107.12,863.0,RIO NACIMIENTO,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,NaN,NaN,NaN,LECHIN,NaN,NaN,NaN,NaN,AL/OL/02,AL/OL/02 NACIMIENTO - ALTO ANDARAX,AL006,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-05-17,2006,23,109,0,0,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-23 14:19:57,520481.06,4111107.12,863.0,RIO NACIMIENTO,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,NaN,NaN,NaN,LECHIN,NaN,NaN,NaN,NaN,AL/OL/02,AL/OL/02 NACIMIENTO - ALTO ANDARAX,AL006,AL006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581788,2021-04-05 17:26:06,008-00013-01-2,sevilla,villanueva de san juan,2021-03-02,2021,8,13,1,2,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-05 17:24:32,303039.00,4104305.00,570.0,LA SIERRA SUR,5.58,18.0,NaN,Franco-arcilloso,Secano,No,NaN,NaN,8 X 8,156.0,2.0,NaN,NaN,HOJIBLANCO,01/03/1990,NaN,NaN,NaN,LAS ATALAYAS,SE/OL/04 SIERRA SUR (OESTE),El Saucejo,El Saucejo,NaN,LAS ATALAYAS,NaN,NaN,NaN,NaN,NaN,SI,Producción Integrada (PI)
581789,2021-04-05 17:26:06,008-0

# 1. Filtering of rows

Taking into account filters of:
- Removing codparcelas null values
- Those that have next measurement inside X days
- Those that the status of the olive does not descend

In [4]:
def find_estado_with_value_two(row) -> int:
    """
    Returns numero del siguiente etado (yt+1). (e.g., 13)
    """
    for column in row.index:
        if row[column] == 2:
            number_growth_stage = int(column.split("_")[-1])
            return number_growth_stage


def get_valid_dataset(df: pd.DataFrame(), max_days_till_next_date: int) -> pd.DataFrame():
    estados = [
    "estado_fenologico_1",
    "estado_fenologico_2",
    "estado_fenologico_3",
    "estado_fenologico_4",
    "estado_fenologico_5",
    "estado_fenologico_6",
    "estado_fenologico_7",
    "estado_fenologico_8",
    "estado_fenologico_9",
    "estado_fenologico_10",
    "estado_fenologico_11",
    "estado_fenologico_12",
    "estado_fenologico_13",
    "estado_fenologico_14",
    ]
    # Removing null and filling empty estados with 0
    df = df.dropna(subset=estados, how="all")
    df = df.dropna(subset=["codparcela"])
    df[estados] = df[estados].fillna(0)

    #Filtering Dataset to keep rows only with one unique 2
    df["count_2s"] = df[estados].eq(2).sum(axis=1)
    df = df[df.count_2s == 1]

    # Sorting by date
    df["fecha"] = pd.to_datetime(df["fecha"])
    df.sort_values(by="fecha", inplace=True)

    # Creating a column to display the number of days till the next observation
    df["next_date"] = df.groupby("codparcela", observed=True)["fecha"].shift(-1)
    df["days_until_next_visit"] = (df["next_date"] - df["fecha"]).dt.days

    # Creating a column to display the next estado_fenológico (yt+1)
    df["estado_actual"] = df[estados].apply(find_estado_with_value_two, axis=1)
    df["next_estado"] = df.groupby("codparcela", observed=True)["estado_actual"].shift(-1)

    # Removing the parcels with only one entry and the last entry for every parcel
    df = df.dropna(subset=["days_until_next_visit"])  # 5150 entries removed

    #Removing entries where estado decreases
    mask_estado_decrease = df["next_estado"] - df["estado_actual"] < 0
    df = df[~mask_estado_decrease]

    #Changing datatypes
    df["days_until_next_visit"] = df["days_until_next_visit"].astype("int16")
    df["next_estado"] = df["next_estado"].astype("int8")
    df[estados] = df[estados].astype("int8")

    # Filtering the max days
    df = df[df["days_until_next_visit"] < max_days_till_next_date]

    excluded_columns = ['estado_actual','count_2s', 'next_date']

    return df.loc[:, [i for i in df.columns if i not in excluded_columns]]

In [5]:
feature_frame=get_valid_dataset(feature_frame, 30)
print(len(feature_frame))

536444


#2. Preprocessing of categorical variables

In total, we find the following categorical variables:
-	Provincia (low cardinality)
-	Municipio (high cardinality)
-	Campaña (Year)
-	05_comarca (medium cardinality)
-	203_orientación (low cardinality)
-	204_textura_del_suelo (medium cardinality)
-	206_secano_/_regadío (binary / low cardinality)
-	12_tipo_de_cubierta_vegetal (low/medium cardinality)
-	304_formación (medium cardinality)
-	308_variedad_principal (medium / high cardinality)
-	120_zona_biológica_raif (high cardinality) (maybe the same as comarca / municipio)
-	310_patrón_variedad_principal (medium cardinality)

Of them, we think the most relevant ones are:
-	Campaña (Year)
-	203_orientación (low cardinality)
-	204_textura_del_suelo (medium cardinality)
-	206_secano_/_regadío (binary / low cardinality)
-	308_variedad_principal (medium / high cardinality)



3. Provincia

In [6]:
print(feature_frame["provincia"].isnull().sum())
counter_provincia=Counter(feature_frame["provincia"])
print(counter_provincia)

0
Counter({'jaen': 149374, 'granada': 127952, 'cordoba': 126910, 'malaga': 61868, 'sevilla': 41768, 'cadiz': 18681, 'huelva': 7352, 'almeria': 2539})


No need for preprocessing

4. Municipio

In [7]:
print(feature_frame["municipio"].isnull().sum())
counter_municipio=Counter(feature_frame["municipio"])
print(counter_municipio)
print(len(counter_municipio))

0
Counter({'ubeda': 23870, 'loja': 17887, 'antequera': 16431, 'iznalloz': 16018, 'puente genil': 15084, 'alcala la real': 13999, 'baena': 13911, 'montefrio': 13228, 'baeza': 7928, 'colomera': 7926, 'adamuz': 7496, 'salar': 6392, 'alameda': 6358, 'mollina': 5776, 'algarinejo': 5580, 'illora': 5517, 'moclin': 5469, 'estepa': 5406, 'navas de san juan': 5293, 'martos': 5284, 'osuna': 5213, 'archidona': 5164, 'olvera': 5138, 'santaella': 5109, 'aguilar': 4928, 'campillos': 4836, 'rute': 4808, 'alcaudete': 4714, 'castro del rio': 4604, 'montillana': 4477, 'priego de cordoba': 4414, 'montejicar': 4346, 'santisteban del puerto': 4102, 'lucena': 4045, 'alhama de granada': 4043, 'pinar': 3958, 'torre del campo': 3891, 'luque': 3694, 'deifontes': 3543, 'castillo de locubin': 3448, 'cabra': 3442, 'torredonjimeno': 3343, 'setenil': 3263, 'cordoba': 3156, 'fuente de piedra': 3148, 'ecija': 3109, 'benalua de las villas': 3014, 'teba': 2936, 'nueva carteya': 2881, 'pozo alcon': 2816, 'bailen': 2759, '

No need for preprocessing

6. Campaña

Campaña should be the year of the measurement, which matches the year in "Fecha" during 99% of the cases.

So, in that 1% where it does not match, we need to trust either "campaña" or "fecha".

As Fecha will be employed as some kind of ID, we will consider it more trustworthy than "Campaña"

The quickest way of implementing this is just deleting "Campaña" and replacing it for the year in Fecha.

In [8]:
#Values coming from Fecha

print(feature_frame["fecha"].isnull().sum())
fecha_separated = pd.DataFrame(columns=['Month','Year'])
fecha_separated['Month'] = feature_frame["fecha"].dt.month
fecha_separated['Year'] = feature_frame["fecha"].dt.year
print(fecha_separated.groupby(['Year']).count()['Month'])

0
Year
2005       79
2006    35988
2007    38156
2008    36029
2009    44677
2010    43986
2011    42426
2012    29397
2013    29211
2014    29465
2015    31063
2016    28571
2017    36801
2018    35038
2019    37226
2020    36385
2021     1946
Name: Month, dtype: int64


In [9]:
#Values coming from Campaña

print(feature_frame["campaña"].isnull().sum())
for key, value in sorted(Counter(feature_frame["campaña"]).items()):
     print(key, value)

0
2005 74
2006 35940
2007 38163
2008 36068
2009 44673
2010 43962
2011 42410
2012 29426
2013 29233
2014 29458
2015 31070
2016 28571
2017 36731
2018 35111
2019 37223
2020 36385
2021 1946


In [10]:
def preprocess_campaña(data_frame):
  data_frame2=data_frame.copy()
  data_frame2.drop(columns=['campaña'])
  data_frame2['campaña']=data_frame2["fecha"].dt.year
  return data_frame2

Check

In [11]:
feature_frame_preprocess_campaña=preprocess_campaña(feature_frame)
for key, value in sorted(Counter(feature_frame_preprocess_campaña["campaña"]).items()):
     print(key, value)

2005 79
2006 35988
2007 38156
2008 36029
2009 44677
2010 43986
2011 42426
2012 29397
2013 29211
2014 29465
2015 31063
2016 28571
2017 36801
2018 35038
2019 37226
2020 36385
2021 1946


30. 105_comarca

We first have to check if one same comarca is not called using multiple names.

Generally, it would not be correct to subdivide a comarca into further parts, as then it would not be a comarca. So, we will implement unifying those comarcas that are subdivided with a I, a II and a III.

In addition, of the null values, the comarcas are added according to the municipio.

In [12]:
provincia_comarca=feature_frame[["provincia","105_comarca"]].drop_duplicates().sort_values(["provincia"])
print(provincia_comarca)

       provincia               105_comarca
223      almeria            RIO NACIMIENTO
2161     almeria            CAMPO TABERNAS
777      almeria            BAJO ALMANZORA
373      almeria            ALTO ALMANZORA
5904       cadiz          CAMPIÑA DE CADIZ
17218      cadiz     SIERRA DE CADIZ - III
3722       cadiz      SIERRA DE CADIZ - II
11073      cadiz           SIERRA DE CADIZ
16817      cadiz     CAMPIÑA DE CADIZ - II
124034   cordoba              CAMPIÑA BAJA
124753   cordoba              LAS COLONIAS
67625    cordoba                       NaN
77821    cordoba                PENIBETICA
88989    cordoba                 LA SIERRA
96738    cordoba                 PEDROCHES
81680    cordoba        CAMPIÑA ALTA - III
32981    cordoba              CAMPIÑA ALTA
67054    cordoba            PEDROCHES - II
106087   cordoba         CAMPIÑA ALTA - II
17515    cordoba            LA SIERRA - II
70121    cordoba           LA SIERRA - III
182440   granada                       NaN
145395   gr

In [13]:
print(feature_frame["105_comarca"].isnull().sum())
counter_comarca=Counter(feature_frame["105_comarca"])
print(counter_comarca)

3558
Counter({'NORTE O ANTEQUERA': 56933, 'LA LOMA': 47967, 'IZNALLOZ': 44354, 'DE LA  VEGA': 35671, 'CAMPIÑA ALTA': 31152, 'CAMPIÑA BAJA': 30713, 'MONTEFRIO': 29762, 'CAMPIÑA DEL SUR': 20805, 'PENIBETICA': 18989, 'ESTEPA': 15536, 'CAMPIÑA ALTA - II': 14765, 'SIERRA SUR - II': 13999, 'SIERRA SUR': 13990, 'LA CAMPIÑA': 13403, 'EL CONDADO': 12762, 'CAMPIÑA DEL NORTE - II': 9473, 'MAGINA': 8635, 'SIERRA DE CADIZ': 8570, 'ALHAMA': 8400, 'LA SIERRA SUR': 7719, 'LA SIERRA - II': 7496, 'SIERRA DE CADIZ - II': 6563, 'LA SIERRA': 6451, 'SIERRA DE SEGURA': 6135, 'PEDROCHES': 6127, 'SIERRA DE CAZORLA': 5776, 'LA SIERRA - III': 5044, nan: 3558, 'HUESCAR': 3453, 'LA SIERRA SUR - II': 3428, 'CONDADO CAMPIÑA': 3385, 'LAS COLONIAS': 3049, 'SERRANIA DE RONDA - II': 2597, 'CAMPIÑA DE CADIZ': 2578, 'CONDADO LITORAL': 2003, 'CAMPIÑA ALTA - III': 1994, 'SIERRA MORENA': 1934, 'MAGINA - II': 1894, 'CAMPIÑA DEL NORTE': 1829, 'COSTA - II': 1778, 'SIERRA DE CAZORLA - II': 1678, 'VALLE DE LECRIN': 1511, 'CAMPO T

In [14]:
#municipios will null comarcas

feature_frame.loc[feature_frame["105_comarca"].isnull()]["municipio"].drop_duplicates()

374612                 antequera
407696                   mollina
336915            torredonjimeno
67625             fuente palmera
122435                 santaella
70065                guadalcazar
91272              palma del rio
78385                      luque
182440                    jayena
232853          villanueva mesia
173253                  iznalloz
223550                     pinar
162423                   escuzar
146246                campotejar
201809                 montefrio
72408                    iznajar
73825                     lucena
95871                 pozoblanco
89139                      obejo
338127              torreperogil
341757                     ubeda
403357          fuente de piedra
212812                montejicar
104312              puente genil
26090                    aguilar
420031    villanueva del trabuco
389671                 archidona
417521    villanueva de algaidas
163362              guadahortuna
419584    villanueva del rosario
327990    

In [15]:
def preprocess_comarca(data_frame):
  data_frame2=data_frame.copy()
  #Sorting out the comarcas with I, II or III.
  replacements_comarcas = {"105_comarca": {"SIERRA DE CADIZ - II": "SIERRA DE CADIZ",
                                  "SIERRA DE CADIZ - III": "SIERRA DE CADIZ",
                                  "CAMPIÑA DE CADIZ - II": "CAMPIÑA DE CADIZ",
                                  "CAMPIÑA ALTA - III": "CAMPIÑA ALTA",
                                  "CAMPIÑA ALTA - II": "CAMPIÑA ALTA",
                                  "LA SIERRA - III": "LA SIERRA",
                                  "LA SIERRA - II": "LA SIERRA",
                                  "PEDROCHES - II": "PEDROCHES",
                                  "GUADIX - II": "GUADIX",
                                  "COSTA - II": "COSTA",
                                  "SIERRA MORENA - II": "SIERRA MORENA",
                                  "SIERRA SUR - II": "SIERRA SUR",
                                  "SIERRA DE CAZORLA - II": "SIERRA DE CAZORLA",
                                  "SIERRA DE CAZORLA - III": "SIERRA DE CAZORLA",
                                  "CAMPIÑA DEL NORTE - III": "CAMPIÑA DEL NORTE",
                                  "CAMPIÑA DEL NORTE - II": "CAMPIÑA DEL NORTE",
                                  "MAGINA - II": "MAGINA",
                                  "SERRANIA DE RONDA - II": "SERRANIA DE RONDA",
                                  "LA SIERRA NORTE - II": "LA SIERRA NORTE",
                                  "LA SIERRA SUR - II": "LA SIERRA SUR",
                                  }}
  data_frame2.replace(replacements_comarcas, inplace=True)

  #Sort comarcas of some municipios.
  replacement_municipios={"antequera":"NORTE O ANTEQUERA",
                          "mollina":"NORTE O ANTEQUERA",
                          "torredonjimeno":"CAMPIÑA DEL SUR",
                          "fuente palmera":"LAS COLONIAS",
                          "santaella":"CAMPIÑA BAJA",
                          "guadalcazar":"LAS COLONIAS",
                          "palma del rio":"CAMPIÑA BAJA",
                          "luque":"PENIBETICA",
                          "jayena":"ALHAMA",
                          "villanueva mesia":"DE LA  VEGA",
                          "iznalloz":"IZNALLOZ",
                          "pinar":"IZNALLOZ",
                          "escuzar":"ALHAMA",
                          "campotejar":"IZNALLOZ",
                          "montefrio":"MONTEFRIO",
                          "iznajar":"PENIBETICA",
                          "lucena":"CAMPIÑA ALTA",
                          "pozoblanco":"PEDROCHES",
                          "obejo":"LA SIERRA",
                          "torreperogil":"LA LOMA",
                          "ubeda":"LA LOMA",
                          "fuente de piedra":"NORTE O ANTEQUERA",
                          "montejicar":"IZNALLOZ",
                          "puente genil":"CAMPIÑA ALTA",
                          "aguilar":"CAMPIÑA ALTA",
                          "villanueva del trabuco":"NORTE O ANTEQUERA",
                          "archidona":"NORTE O ANTEQUERA",
                          "villanueva de algaidas":"NORTE O ANTEQUERA",
                          "guadahortuna":"IZNALLOZ",
                          "villanueva del rosario":"NORTE O ANTEQUERA",
                          "santisteban del puerto":"EL CONDADO",
                          "navas de san juan":"EL CONDADO",
                          "montillana":"IZNALLOZ",
                          "villacarrillo":"LA LOMA",
                          "iznatoraf":"LA LOMA",
                          "alcaudete":"CAMPIÑA DEL SUR",
                          "priego de cordoba":"PENIBETICA",
                          "jimena":"MAGINA",
                          "torreblascopedro":"CAMPIÑA DEL NORTE"}
  data_frame2["105_comarca"].fillna(data_frame2['municipio'].map(replacement_municipios), inplace=True)
  return data_frame2

Checks

In [16]:
feature_frame_preprocess_comarcas=preprocess_comarca(feature_frame)
print(feature_frame_preprocess_comarcas[["provincia","105_comarca"]].drop_duplicates().sort_values(["provincia"]))

       provincia               105_comarca
223      almeria            RIO NACIMIENTO
2161     almeria            CAMPO TABERNAS
777      almeria            BAJO ALMANZORA
373      almeria            ALTO ALMANZORA
5904       cadiz          CAMPIÑA DE CADIZ
11073      cadiz           SIERRA DE CADIZ
96738    cordoba                 PEDROCHES
32981    cordoba              CAMPIÑA ALTA
124034   cordoba              CAMPIÑA BAJA
124753   cordoba              LAS COLONIAS
77821    cordoba                PENIBETICA
88989    cordoba                 LA SIERRA
145476   granada               DE LA  VEGA
146597   granada                   HUESCAR
159526   granada         MONTES ORIENTALES
132817   granada                 MONTEFRIO
161297   granada                    GUADIX
139087   granada                    ALHAMA
142323   granada                      BAZA
225653   granada                  IZNALLOZ
161907   granada           VALLE DE LECRIN
145395   granada            LAS ALPUJARRAS
240995    h

In [17]:
print(feature_frame_preprocess_comarcas["105_comarca"].isnull().sum())

0


34. 203_orientación

We check the null values according to the landplot.

The orientations means to which direction does the slope go. There would be in principle the 8 cardinal points:
- N
- NE
- E
- SE
- S
- SO
- O
- NO
- Flat landplots

So, we do as follows:
1. Check if other parcelas have the same values and copy across them.
2. Homogeneize the values
3. For the remaining null values, if 202_pendiente_(%) is null then it is introduced to the flat category; if 202_pendiente_(%) is not null, then introduced to category "not defined".

In [18]:
print(feature_frame["203_orientación"].isnull().sum())
counter_orientacion=Counter(feature_frame["203_orientación"])
print(counter_orientacion)

310032
Counter({nan: 310032, 'N': 21286, 'S': 21157, '2 - NE': 18587, '1 - N': 16104, '4 - SE': 15661, '5 - S': 15184, 'SE': 14311, 'NE': 11592, '3 - E': 10323, 'NO': 10266, '0 - Llana': 9498, '8 - NO': 9315, 'E': 9042, '6 - SO': 8836, '7 - O': 7976, '9 - Varias': 6374, 'O': 5964, 'SO': 5526, 'LLANO': 2570, 'N-S': 1393, 'X': 1327, 'EO': 714, '1/2 N 1/2 S': 685, 'LL': 577, 'LLANA': 488, 'ESTE': 362, 'NS': 276, 'E-NE': 170, 'E-O': 160, 'SIN DEFINIR': 139, 'SUROESTE': 136, 'N-E': 128, 'llano': 76, 'SUR ESTE': 71, 'SURESTE': 71, 'N-O': 67})


In [19]:
def preprocess_orientacion(data_frame):

  data_frame2=data_frame.copy()

  #Check for the parcelas that might have both null values and an orientation
  dataframe_nonnull_perparcela=data_frame2.groupby('codparcela')["203_orientación"].first().dropna().to_dict()
  for index, row in data_frame2.iterrows():
    if row["codparcela"] in dataframe_nonnull_perparcela:
      data_frame2.at[index, "203_orientación"] = dataframe_nonnull_perparcela[row["codparcela"]]

  #Sorting out homogeneization.
  replacements_orientacion = {"203_orientación": {"2 - NE": "NE",
                                  "1 - N": "N",
                                  "4 - SE": "SE",
                                  "5 - S": "S",
                                  "3 - E": "E",
                                  "0 - Llana": "LLANO",
                                  "8 - NO": "NO",
                                  "6 - SO": "SO",
                                  "7 - O": "O",
                                  "9 - Varias": "SIN DEFINIR",
                                  "N-S": "SIN DEFINIR",
                                  "X": "SIN DEFINIR",
                                  "EO": "SIN DEFINIR",
                                  "1/2 N 1/2 S": "SIN DEFINIR",
                                  "LL": "LLANO",
                                  "LLANA": "LLANO",
                                  "ESTE": "E",
                                  "NS": "SIN DEFINIR",
                                  "E-NE": "NE",
                                  "E-O": "SIN DEFINIR",
                                  "SUROESTE": "SO",
                                  "N-E": "NE",
                                  "llano": "LLANO",
                                  "SUR ESTE": "SE",
                                  "SURESTE": "SE",
                                  "N-O": "NO",
                                  }}
  data_frame2.replace(replacements_orientacion, inplace=True)

  #Null values in pendiente means LLano, non null values in pendiente means No definido
  for index, row in data_frame2.iterrows():
    if row["203_orientación"] is np.nan:
      if row["202_pendiente_(%)"] is np.nan:
        data_frame2.at[index, "203_orientación"] = "LLANO"
      else:
        data_frame2.at[index, "203_orientación"] = "SIN DEFINIR"

  return data_frame2

Checks

In [20]:
feature_frame_preprocess_orientacion=preprocess_orientacion(feature_frame)
print(feature_frame_preprocess_orientacion["203_orientación"].isnull().sum())

0


In [21]:
counter_orientacion=Counter(feature_frame_preprocess_orientacion["203_orientación"])
print(counter_orientacion)

Counter({'SIN DEFINIR': 295994, 'N': 40715, 'S': 39211, 'NE': 36987, 'SE': 33248, 'NO': 22554, 'E': 20573, 'SO': 16525, 'O': 16233, 'LLANO': 14404})


34. 204_textura_del_suelo

We check the null values according to the landplot.

The textura means the kind of land we have:

- Franco
- Arcilloso
- Limoso
- Arenoso
- Franco-Arcilloso
- Franco-Arenoso
- Franco-Limoso
- Arcillo-Arenoso
- Arcillo-Limoso
- Limoso-Arenoso
- Sin definir

Same philosophy as before:
1. Check if other parcelas have the same values and copy across them.
2. Homogeneize the values
3. For the remaining null values, add to not defined.

In [22]:
print(feature_frame["204_textura_del_suelo"].isnull().sum())
counter_textura=Counter(feature_frame["204_textura_del_suelo"])
print(counter_textura)

292643
Counter({nan: 292643, 'FRANCO-ARCILLOSO': 47827, 'ARCILLO-ARENOSO': 24502, 'FRANCO': 22862, 'FRANCO-ARENOSO': 20884, 'ARCILLOSO': 19198, 'Franco-arcilloso': 16169, 'FRANCO-ARCILLO-ARENOSO': 12328, 'Calizas y margas (localmente areniscas o radiolari': 9077, 'ARCILLO-LIMOSO': 8990, 'FRANCO-ARCILLO-LIMOSO': 7026, 'ARENO-FRANCO': 6575, 'Franco': 4931, 'Arcilloso': 4776, 'Areno-franco': 4754, 'Conglomerados, arenas, lutitas y calizas': 4305, 'Margas, margocalizas, calizas (localmente calcaren': 4170, 'ARENOSO': 3856, 'Franco-arcillo-arenoso': 3310, 'FRANCO-LIMOSO': 2690, 'Calcarenitas, arenas, margas y calizas': 2348, 'Franco-arenoso': 2279, 'Margas yesíferas, areniscas y calizas': 1939, 'MEDIA': 1187, 'Margas, areniscas y lutitas o silexitas': 1125, 'Franco-arcillo-limoso': 1004, 'Arcillo-arenoso': 796, 'FRANCO ARCILLOSO': 779, 'Arenas, limos, arcillas, gravas y cantos': 647, 'Limoso': 561, 'Arenoso': 373, 'FRANCO CON ELEMENTOS GRUESOS': 372, 'Franco-Arenoso-Arcilloso': 362, 'Margas

In [25]:
def preprocess_textura(data_frame):

  data_frame2=data_frame.copy()

  #Check for the parcelas that might have both null values and an orientation
  dataframe_nonnull_perparcela=data_frame2.groupby('codparcela')["204_textura_del_suelo"].first().dropna().to_dict()
  for index, row in data_frame2.iterrows():
    if row["codparcela"] in dataframe_nonnull_perparcela:
      data_frame2.at[index, "204_textura_del_suelo"] = dataframe_nonnull_perparcela[row["codparcela"]]

  #Sorting out homogeneization.
  replacements_orientacion = {"204_textura_del_suelo": {"Franco-arcilloso": "FRANCO-ARCILLOSO",
                                  "FRANCO-ARCILLO-ARENOSO": "FRANCO",
                                  "Calizas y margas (localmente areniscas o radiolari": "FRANCO",
                                  "FRANCO-ARCILLO-LIMOSO": "FRANCO",
                                  "ARENO-FRANCO": "FRANCO-ARENOSO",
                                  "Franco": "FRANCO",
                                  "Arcilloso": "ARCILLOSO",
                                  "Areno-franco": "FRANCO-ARENOSO",
                                  "Conglomerados, arenas, lutitas y calizas": "ARCILLO-ARENOSO",
                                  "Margas, margocalizas, calizas (localmente calcaren": "FRANCO",
                                  "Franco-arcillo-arenoso": "FRANCO",
                                  "Calcarenitas, arenas, margas y calizas": "FRANCO-ARENOSO",
                                  "Franco-arenoso": "FRANCO-ARENOSO",
                                  "Margas yesíferas, areniscas y calizas": "FRANCO-ARENOSO",
                                  "MEDIA": "SIN DEFINIR",
                                  "Margas, areniscas y lutitas o silexitas": "FRANCO-ARENOSO",
                                  "Franco-arcillo-limoso": "FRANCO",
                                  "Arcillo-arenoso": "ARCILLO-ARENOSO",
                                  "FRANCO ARCILLOSO": "FRANCO-ARCILLOSO",
                                  "Arenas, limos, arcillas, gravas y cantos": "ARENO-LIMOSO",
                                  "Limoso": "LIMOSO",
                                  "Arenoso": "ARENOSO",
                                  "FRANCO CON ELEMENTOS GRUESOS": "FRANCO",
                                  "Franco-Arenoso-Arcilloso": "FRANCO",
                                  "Margas, margocalizas, calizas(localmente calcaren)": "FRANCO",
                                  "Calizas y margas (localmente arenisca": "FRANCO-ARENOSO",
                                  "Arcillo-limoso": "ARCILLO-LIMOSO",
                                  "Areniscas, margas y lutitas": "FRANCO-ARENOSO",
                                  "FRANCO LIMOSO": "FRANCO-LIMOSO",
                                  "FRANCO - ARCILLO - ARENOSO": "FRANCO",
                                  "FRANCO-ARC": "FRANCO-ARCILLOSO",
                                  "Franco arcilloso": "FRANCO-ARCILLOSO",
                                  "FRANCO - ARENOSO": "FRANCO-ARENOSO",
                                  "calizo": "ARENOSO",
                                  }}
  data_frame2.replace(replacements_orientacion, inplace=True)

  #Null values in pendiente means LLano, non null values in pendiente means No definido

  data_frame2["204_textura_del_suelo"] = data_frame2["204_textura_del_suelo"].fillna("SIN DEFINIR")
  """
  for index, row in data_frame2.iterrows():
    if row["204_textura_del_suelo"] is np.nan:
      data_frame2.at[index, "204_textura_del_suelo"] = "SIN DEFINIR"
  """

  return data_frame2

Checks

In [26]:
feature_frame_preprocess_textura=preprocess_textura(feature_frame)
print(feature_frame_preprocess_textura["204_textura_del_suelo"].isnull().sum())

0


In [27]:
counter_textura=Counter(feature_frame_preprocess_textura["204_textura_del_suelo"])
print(counter_textura)

Counter({'SIN DEFINIR': 257641, 'FRANCO-ARCILLOSO': 83026, 'FRANCO-ARENOSO': 60169, 'FRANCO': 60104, 'ARCILLOSO': 28552, 'ARCILLO-ARENOSO': 28421, 'ARCILLO-LIMOSO': 9749, 'ARENOSO': 4767, 'FRANCO-LIMOSO': 3340, 'LIMOSO': 550, 'ARENO-LIMOSO': 125})


35. 206_secano_/_regadío

We check the null values according to the landplot.

The secano/regadio means how we water the land:

- Secano
- Regadio
- Riego de apoyo

Same philosophy as before:

1. Check if other parcelas have the same values and copy across them.
2. Homogeneize the values
3. For the remaining null values, add to not defined.

In [28]:
print(feature_frame["206_secano_/_regadío"].isnull().sum())
counter_secano_regadio=Counter(feature_frame["206_secano_/_regadío"])
print(counter_secano_regadio)

62789
Counter({'SECANO': 212267, 'REGADÍO': 123231, 'Secano': 70679, nan: 62789, 'Regadío': 41731, 'RIEGO DE APOYO': 18403, 'Riego de apoyo': 4397, 'REGADIO': 2107, 'SECANO CON RIEGO DE APOYO': 348, 'RIEGO': 337, 'RlEGO': 116, 'Regadio': 39})


In [29]:
def preprocess_riego(data_frame):

  data_frame2=data_frame.copy()

  #Check for the parcelas that might have both null values and an secano/regadio
  dataframe_nonnull_perparcela=data_frame2.groupby('codparcela')["206_secano_/_regadío"].first().dropna().to_dict()
  for index, row in data_frame2.iterrows():
    if row["codparcela"] in dataframe_nonnull_perparcela:
      data_frame2.at[index, "206_secano_/_regadío"] = dataframe_nonnull_perparcela[row["codparcela"]]

  #Sorting out homogeneization.
  replacements_orientacion = {"206_secano_/_regadío": {"REGADÍO": "REGADIO",
                                  "Secano": "SECANO",
                                  "Regadío": "REGADIO",
                                  "Riego de apoyo": "RIEGO DE APOYO",
                                  "SECANO CON RIEGO DE APOYO": "RIEGO DE APOYO",
                                  "RIEGO": "SIN DEFINIR",
                                  "RlEGO": "SIN DEFINIR",
                                  "Regadio": "REGADIO",
                                  }}
  data_frame2.replace(replacements_orientacion, inplace=True)

  #Null values in pendiente means LLano, non null values in pendiente means No definido

  data_frame2["206_secano_/_regadío"] = data_frame2["206_secano_/_regadío"].fillna("SIN DEFINIR")
  """
  for index, row in data_frame2.iterrows():
    if row["206_secano_/_regadío"] is np.nan:
      data_frame2.at[index, "206_secano_/_regadío"] = "SIN DEFINIR"
  """


  return data_frame2

Checks

In [30]:
feature_frame_preprocess_riego=preprocess_riego(feature_frame)
print(feature_frame_preprocess_riego["206_secano_/_regadío"].isnull().sum())

0


In [31]:
counter_riego=Counter(feature_frame_preprocess_riego["206_secano_/_regadío"])
print(counter_riego)

Counter({'SECANO': 305748, 'REGADIO': 176728, 'SIN DEFINIR': 28576, 'RIEGO DE APOYO': 25392})


37. 212_tipo_de_cubierta_vegetal

We check the null values according to the landplot.

The cubierta vegetal refers to how the landplot is covered:

- Silvestre
- Restos poda
- Sembrada
- Sin definir

Same philosophy as before:

1. Check if other parcelas have the same values and copy across them.
2. Homogeneize the values
3. For the remaining null values, add to not defined.

In [32]:
print(feature_frame["212_tipo_de_cubierta_vegetal"].isnull().sum())
counter_tipocubiertavegetal=Counter(feature_frame[feature_frame["212_tipo_de_cubierta_vegetal"].notnull()]["212_tipo_de_cubierta_vegetal"])
print(counter_tipocubiertavegetal)

301766
Counter({'Silvestre': 150457, 'SILVESTRE FORMADA CON HERBICIDA': 38488, 'ESPONTANEA': 13835, 'Inerte ( restos poda, ..)': 12449, 'SILVESTRE FORMADA CON LABORES': 7810, 'Instalada con siembra': 2613, 'INSTALADA CON SIEMBRA': 2155, 'INERTE (RESTOS DE PODA)': 1304, 'NO': 1215, 'NATURAL': 1108, 'NINGUNA': 853, 'SILVESTRE': 365, 'CONTROLADA CON GANADO': 342, 'RESTOS VEGETALES': 322, 'SILVESTRE DESBROZADORA (PARCELAS)': 259, 'SILVESTRE FORMADA CON HERBICIDA Y RESTOS DE PODA': 210, 'PARCIAL': 161, 'PIEDRA NATURAL': 158, 'SIN DEFINIR': 139, 'SI': 71, 'EXPONTANEA': 71, 'INSTALADA CON HERBICIDAS': 71, 'RESTOS PODA': 56, 'CUBIERTA SILVESTRE CON DESBROZADORA': 53, 'SILVESTRE CONTROLADA POR PASTOREO': 39, 'SE INSTALARA EN SEPTIENBRE 2006': 27, 'SILVESTRE FORMADA CON GANADO OVINO': 24, 'FLORA NATURAL': 23})


In [33]:
def preprocess_cubierta(data_frame):

  data_frame2=data_frame.copy()

  #Check for the parcelas that might have both null values and an secano/regadio
  dataframe_nonnull_perparcela=data_frame2.groupby('codparcela')["212_tipo_de_cubierta_vegetal"].first().dropna().to_dict()
  for index, row in data_frame2.iterrows():
    if row["codparcela"] in dataframe_nonnull_perparcela:
      data_frame2.at[index, "212_tipo_de_cubierta_vegetal"] = dataframe_nonnull_perparcela[row["codparcela"]]

  #Sorting out homogeneization.
  replacements_orientacion = {"212_tipo_de_cubierta_vegetal": {"Silvestre": "SILVESTRE",
                                  "SILVESTRE FORMADA CON HERBICIDA": "SILVESTRE",
                                  "ESPONTANEA": "SILVESTRE",
                                  "Inerte ( restos poda, ..)": "RESTOS PODA",
                                  "SILVESTRE FORMADA CON LABORES": "SILVESTRE",
                                  "Instalada con siembra": "SIEMBRA",
                                  "INSTALADA CON SIEMBRA": "SIEMBRA",
                                  "INERTE (RESTOS DE PODA)": "RESTOS PODA",
                                  "NO": "SIN DEFINIR",
                                  "NATURAL": "SILVESTRE",
                                  "CONTROLADA CON GANADO": "SILVESTRE",
                                  "RESTOS VEGETALES": "RESTOS PODA",
                                  "SILVESTRE DESBROZADORA (PARCELAS)": "SILVESTRE",
                                  "SILVESTRE FORMADA CON HERBICIDA Y RESTOS DE PODA": "SILVESTRE",
                                  "PARCIAL": "SILVESTRE",
                                  "PIEDRA NATURAL": "SIN DEFINIR",
                                  "SI": "SILVESTRE",
                                   "EXPONTANEA": "SILVESTRE",
                                   "INSTALADA CON HERBICIDAS": "SILVESTRE",
                                   "RESTOS PODA": "RESTOS PODA",
                                   "CUBIERTA SILVESTRE CON DESBROZADORA": "SILVESTRE",
                                   "SILVESTRE CONTROLADA POR PASTOREO": "SILVESTRE",
                                  "SE INSTALARA EN SEPTIENBRE 2006": "SIN DEFINIR",
                                  "SILVESTRE FORMADA CON GANADO OVINO": "SILVESTRE",
                                  "FLORA NATURAL": "SILVESTRE",
                                  "NINGUNA": "SIN DEFINIR"
                                  }}
  data_frame2.replace(replacements_orientacion, inplace=True)


  data_frame2["212_tipo_de_cubierta_vegetal"] = data_frame2["212_tipo_de_cubierta_vegetal"].fillna("SIN DEFINIR")
  """
  for index, row in data_frame2.iterrows():
    if row["212_tipo_de_cubierta_vegetal"] is np.nan:
      data_frame2.at[index, "212_tipo_de_cubierta_vegetal"] = "SIN DEFINIR"
  """

  return data_frame2

Checks

In [34]:
feature_frame_preprocess_cubierta=preprocess_cubierta(feature_frame)
print(feature_frame_preprocess_cubierta["212_tipo_de_cubierta_vegetal"].isnull().sum())

0


In [35]:
counter_cubierta=Counter(feature_frame_preprocess_cubierta["212_tipo_de_cubierta_vegetal"])
print(counter_cubierta)

Counter({'SIN DEFINIR': 270623, 'SILVESTRE': 244939, 'RESTOS PODA': 14876, 'SIEMBRA': 6006})


42. 304_formación

We check the null values according to the landplot.

The cubierta vegetal refers to how the olive tree is trimmed:

- Formación
- Mantenimiento
- Rejuvenecimiento
- Sin definir

Same philosophy as before:

1. Check if other parcelas have the same values and copy across them.
2. Homogeneize the values
3. For the remaining null values, add to not defined.

In [36]:
print(feature_frame["304_formación"].isnull().sum())
counter_formacion=Counter(feature_frame["304_formación"])
print(counter_formacion)

391156
Counter({nan: 391156, 'Vaso': 78191, 'VASO': 31953, 'MARCO REAL': 5302, 'REAL': 3424, 'TRADICIONAL': 2517, 'vaso': 1940, 'REJUVENECIMIENTO': 1924, 'Espaldera': 1673, 'TRESBOLILLO': 1515, 'VAS0': 1488, 'Natural': 1422, 'FORMACION': 1274, 'PODA DE RENOVACIÓN': 1271, 'EN VASO LIBRE': 1088, 'NATURAL': 985, 'PRODUCCION Y REJUVENECIMIENTO': 907, 'MARCO TRESBOLILLO': 905, 'LIBRE': 700, 'cuadro': 541, '3 PIES': 531, 'De Producción': 409, 'PODA DE FORMACIÓN': 355, 'tresbolillo': 350, 'ENBASO': 320, 'TRADICIONAL VARIOS PIES': 319, 'PODA JAEN': 314, '2 PIES': 301, 'FORMACION Y PRODUCCION': 292, 'FORMACION Y REJUVENECIMIENTO': 285, 'VARIABLE': 278, 'VARIOS PIES': 207, '3 pies': 181, 'PALANCAS 1 PIE': 176, 'PLANTACIÓN IRREGULAR': 158, 'CINCO DE OROS': 152, 'ALEATORIA': 150, '10 METROS': 147, 'rectangulo': 145, 'SIN DEFINIR': 139, 'PODA DE RENOVACION': 138, 'Rejuvenecimiento': 127, 'TRADICIONAL 3-4 PIES': 122, 'GLOBO': 121, 'UN PIE': 112, 'INTENSIVO': 65, 'De formación': 61, 'Tradicional': 58

In [37]:
def preprocess_formacion(data_frame):

  data_frame2=data_frame.copy()

  #Check for the parcelas that might have both null values and an secano/regadio
  dataframe_nonnull_perparcela=data_frame2.groupby('codparcela')["304_formación"].first().dropna().to_dict()
  for index, row in data_frame2.iterrows():
    if row["codparcela"] in dataframe_nonnull_perparcela:
      data_frame2.at[index,"304_formación"] = dataframe_nonnull_perparcela[row["codparcela"]]

  #Sorting out homogeneization.
  replacements_orientacion = {"304_formación": {"Vaso": "FORMACION",
                                  "VASO": "FORMACION",
                                  "MARCO REAL": "FORMACION",
                                  "TRADICIONAL": "FORMACION",
                                  "vaso": "FORMACION",
                                  "Espaldera": "MANTENIMIENTO",
                                  "TRESBOLILLO": "FORMACION",
                                  "VAS0": "FORMACION",
                                  "Natural": "FORMACION",
                                  "PODA DE RENOVACIÓN": "MANTENIMIENTO",
                                  "EN VASO LIBRE": "FORMACION",
                                  "NATURAL": "FORMACION",
                                  "PRODUCCION Y REJUVENECIMIENTO": "REJUVENECIMIENTO",
                                  "MARCO TRESBOLILLO": "FORMACION",
                                  "LIBRE": "FORMACION",
                                  "cuadro": "FORMACION",
                                  "3 PIES": "FORMACION",
                                   "De Producción": "MANTENIMIENTO",
                                   "PODA DE FORMACIÓN": "FORMACION",
                                   "tresbolillo": "FORMACION",
                                   "ENBASO": "FORMACION",
                                   "TRADICIONAL VARIOS PIES": "FORMACION",
                                  "PODA JAEN": "REJUVENECIMIENTO",
                                  "2 PIES": "FORMACION",
                                  "FORMACION Y PRODUCCION": "MANTENIMIENTO",
                                  "FORMACION Y REJUVENECIMIENTO": "REJUVENECIMIENTO",
                                  "VARIABLE":"SIN DEFINIR",
                                  "VARIOS PIES":"SIN DEFINIR",
                                  "3 OROS":"SIN DEFINIR",
                                  "ALEATORIA":"FORMACION",
                                  "10 METROS":"SIN DEFINIR",
                                  "rectangulo":"SIN DEFINIR",
                                   "PODA DE RENOVACION":"REJUVENECIMIENTO",
                                   "Rejuvenecimiento":"REJUVENECIMIENTO",
                                      "TRADICIONAL 3-4 PIES":"SIN DEFINIR",
                                      "GLOBO":"FORMACION",
                                      "UN PIE":"SIN DEFINIR",
                                       "INTENSIVO":"MANTENIMIENTO",
                                         "De formación":"FORMACION",
                                         "Tradicional":"FORMACION",
                                       "ESTACAS A 3 PIES":"SIN DEFINIR",
                                         "GLOBOSA":"FORMACION",
                                           "Globosa":"FORMACION",
                                           "COPA":"FORMACION",
                                           "GARROTE TRADICIONAL":"SIN DEFINIR",
                                           "Estacas a 3 pies":"SIN DEFINIR",
                                           "ESTACAS A 2 PIES":"SIN DEFINIR",
                                            "PALANCAS 1 PIE":"SIN DEFINIR",
                                            "REAL":"FORMACION",
                                            "3 pies":"SIN DEFINIR",
                                            "PLANTACIÓN IRREGULAR":"SIN DEFINIR",
                                            "CINCO DE OROS":"SIN DEFINIR",
                                  }}
  data_frame2.replace(replacements_orientacion, inplace=True)

  data_frame2["304_formación"] = data_frame2["304_formación"].fillna("SIN DEFINIR")
  """
  for index, row in data_frame2.iterrows():
    if row["304_formación"] is np.nan:
      data_frame2.at[index, "304_formación"] = "SIN DEFINIR"
  """
  return data_frame2

Checks

In [38]:
feature_frame_preprocess_formacion=preprocess_formacion(feature_frame)
print(feature_frame_preprocess_formacion["304_formación"].isnull().sum())

0


In [39]:
counter_formacion=Counter(feature_frame_preprocess_formacion["304_formación"])
print(counter_formacion)

Counter({'SIN DEFINIR': 371885, 'FORMACION': 154260, 'REJUVENECIMIENTO': 6017, 'MANTENIMIENTO': 4282})


44. 308_variedad_principal

We check the null values according to the landplot.

The variedad principal refers to the type of olive we have. Most of them have 2 names (picual == marteno, hojiblanca == Lucentino...):

- Picual
- Hojiblanca
- Manzanilla
- Arbequina
- Lechin
- Picudo
- Nevadillo negro
- Cornicabra
- Lucio
- Gordal
- Verdial
- Sin definir

Same philosophy as before:

1. Check if other parcelas have the same values and copy across them.
2. Homogeneize the values
3. For the remaining null values, add to not defined.

In [40]:
print(feature_frame["308_variedad_principal"].isnull().sum())
counter_variedad=Counter(feature_frame["308_variedad_principal"])
print(counter_variedad)

51969
Counter({'Picual, Marteño': 169989, 'Hojiblanca, Lucentino': 86221, 'PICUAL / MARTEÑO': 84425, nan: 51969, 'HOJIBLANCO': 39318, 'PICUAL': 29206, 'Manzanilla de Sevilla': 10853, 'Arbequina': 9319, 'Lechín de Sevilla, Zorzaleño, Ecijano': 6582, 'Picudo': 5987, 'MANZANILLO': 5082, 'PICUDO': 4985, 'Nevadillo Negro': 4552, 'LECHIN': 3601, 'HOJIBLANCA': 3141, 'NEVADILLO BLANCO': 2971, 'Picual': 2742, 'ARBEQUINA': 1997, 'Gordal Sevillana': 1959, 'VERDIAL': 1688, 'Cornicabra': 734, 'Verdial de Huévar': 669, 'NEVADILLO': 668, 'PICUAL Y PICUDO': 636, 'Lucio': 633, 'Verdial de Vélez-Málaga': 626, 'ALOREÑA': 593, 'PICUDO/PICUAL': 505, 'PICUAL/MARTEÑO': 478, 'Aceite+Mesa': 457, 'Lechín de Sevilla': 452, 'MARTEÑO': 429, 'LUCIO': 345, 'MARTEÑA': 291, 'ARBEQUINA (15000 OLIVOS)': 267, 'MANZ. ZAHURD 87-C.ALIZN 72-COTO 86- DETR. NAVE 98': 266, 'Royal de Cazorla': 161, 'PICUDO Y MARTEÑO': 160, 'Hojiblanca': 152, 'Manzanilla Cacereña': 146, 'Lechín de Granada, Cuquillo': 145, 'PICUDO/HOJIBLANCO': 140

In [41]:
def preprocess_varprinc(data_frame):

  data_frame2=data_frame.copy()

  #Check for the parcelas that might have both null values and an secano/regadio
  dataframe_nonnull_perparcela=data_frame2.groupby('codparcela')["308_variedad_principal"].first().dropna().to_dict()
  for index, row in data_frame2.iterrows():
    if row["codparcela"] in dataframe_nonnull_perparcela:
      data_frame2.at[index,"308_variedad_principal"] = dataframe_nonnull_perparcela[row["codparcela"]]

  #Sorting out homogeneization.
  replacements_orientacion = {"308_variedad_principal": {"Picual, Marteño": "PICUAL",
                                  "Hojiblanca, Lucentino": "HOJIBLANCA",
                                  "PICUAL / MARTEÑO": "PICUAL",
                                  "HOJIBLANCO": "HOJIBLANCA",
                                  "Manzanilla de Sevilla": "MANZANILLA",
                                  "Picudo": "PICUDO",
                                  "Arbequina": "ARBEQUINA",
                                  "Lechín de Sevilla, Zorzaleño, Ecijano": "LECHIN",
                                  "MANZANILLO": "MANZANILLA",
                                  "Nevadillo Negro": "NEVADILLO NEGRO",
                                  "NEVADILLO BLANCO": "PICUAL",
                                  "Picual": "PICUAL",
                                  "Gordal Sevillana": "GORDAL",
                                  "Cornicabra": "CORNICABRA",
                                  "Verdial de Huévar": "VERDIAL",
                                  "NEVADILLO": "PICUAL",
                                  "PICUAL Y PICUDO": "PICUAL",
                                   "Lucio": "LUCIO",
                                   "Verdial de Vélez-Málaga": "VERDIAL",
                                   "ALOREÑA": "MANZANILLA",
                                   "PICUDO/PICUAL": "PICUDO",
                                   "PICUAL/MARTEÑO": "PICUAL",
                                  "Aceite+Mesa": "SIN DEFINIR",
                                  "Lechín de Sevilla": "LECHIN",
                                  "MARTEÑO": "PICUAL",
                                  "MARTEÑA": "PICUAL",
                                  "ARBEQUINA (15000 OLIVOS)":"ARBEQUINA",
                                  "MANZ. ZAHURD 87-C.ALIZN 72-COTO 86- DETR. NAVE 98":"MANZANILLA",
                                  "Royal de Cazorla":"SIN DEFINIR",
                                  "PICUDO Y MARTEÑO":"PICUDO",
                                  "Hojiblanca":"HOJIBLANCA",
                                  "Manzanilla Cacereña":"MANZANILLA",
                                  "Lechín de Granada, Cuquillo":"LECHIN",
                                  "PICUDO/HOJIBLANCO":"PICUDO",
                                  "Manzanilla de Huelva":"MANZANILLA",
                                  "Picual de Almería":"PICUAL",
                                  "VIDUEÑO":"SIN DEFINIR",
                                  "Mollar de Cieza":"SIN DEFINIR",
                                  "Frantoio":"SIN DEFINIR",
                                  "HOJIBLAANCO":"HOJIBLANCA",
                                  "Morona":"SIN DEFINIR",
                                  "Arbequino":"ARBEQUINA",
                                  "PICUAL, MARTEÑO":"PICUAL",
                                  "HOJIBLANCO/ARBEQUINO":"HOJIBLANCA",
                                  "Ocal":"SIN DEFINIR",
                                  "ZARZALEÑO":"LECHIN",
                                  "OTRAS":"SIN DEFINIR",
                                  "Pajarero":"SIN DEFINIR",
                                  "Gordal de Granada":"SIN DEFINIR",
                                  "Arbosana":"SIN DEFINIR",
                                  }}
  data_frame2.replace(replacements_orientacion, inplace=True)

  data_frame2["308_variedad_principal"] = data_frame2["308_variedad_principal"].fillna("SIN DEFINIR")

  """
  for index, row in data_frame2.iterrows():
    if row["308_variedad_principal"] is np.nan:
      data_frame2.at[index, "308_variedad_principal"] = "SIN DEFINIR"
  """
  return data_frame2

Checks

In [42]:
feature_frame_preprocess_varprinc=preprocess_varprinc(feature_frame)
print(feature_frame_preprocess_varprinc["308_variedad_principal"].isnull().sum())

0


In [43]:
counter_varprinc=Counter(feature_frame_preprocess_varprinc["308_variedad_principal"])
print(counter_varprinc)

Counter({'PICUAL': 309143, 'HOJIBLANCA': 135977, 'SIN DEFINIR': 28978, 'MANZANILLA': 17548, 'PICUDO': 11647, 'ARBEQUINA': 11577, 'LECHIN': 9998, 'NEVADILLO NEGRO': 4896, 'VERDIAL': 2734, 'GORDAL': 2114, 'LUCIO': 1098, 'CORNICABRA': 734})


50. 120_zona_biológica_raif

We check the null values according to the landplot.

The zona_biológica_raif is some kind of code:

- There are some of them that contain subdivisions (like a I, II, III...). As each subdivision corresponds to a different zona_biologica_code, we will not homogenise them like we did in the comarcas.


Same philosophy as before:

1. Check if other parcelas have the same values and copy across them.
2. For the remaining null values, add to not defined.

In [44]:
print(feature_frame["120_zona_biológica_raif"].isnull().sum())
counter_zonabiologica=Counter(feature_frame["120_zona_biológica_raif"])
print(counter_zonabiologica)

7303
Counter({'GR/OL/07 IZNALLOZ': 48820, 'JA/OL/12 LOMA BAJA': 44075, 'MA/OL/04 ANTEQUERA-NORTE': 43553, 'GR/OL/01 MONTEFRIO': 40106, 'CO/OL/10 CAMPIÑA ALTA OCCIDENTAL': 27797, 'JA/OL/08 SIERRA AHILLOS Y SAN PEDRO': 23694, 'GR/OL/02 LA VEGA': 19451, 'CO/OL/08 CAMPIÑA ALTA ORIENTAL I': 14511, 'CO/OL/06 CAMPIÑA BAJA OCCIDENTAL': 13483, 'MA/OL/07 ANTEQUERA OCCIDENTAL': 12009, 'JA/OL/02 LOMA ALTA': 11890, 'CO/OL/02 SIERRA MORENA CENTRAL': 11426, 'JA/OL/13 CAMPIÑA SUR': 10471, 'CO/OL/09 CAMPIÑA ALTA ORIENTAL II': 10390, 'SE/OL/01 ESTEPA NORTE': 9625, 'GR/OL/03 ALHAMA DE GRANADA': 9262, 'JA/OL/01 SIERRA MORENA': 8716, 'SE/OL/08 CAMPIÑA': 8551, 'JA/OL/05 SIERRA SUR': 8466, 'JA/OL/15 SIERRA CAZORLA': 8067, nan: 7303, 'CO/OL/11 SUBBÉTICA SEPTENTRIONAL': 7192, 'JA/OL/14 CAMPIÑA NORTE': 7079, 'CA/OL/04 OLVERA': 6968, 'JA/OL/06 SIERRA SEGURA': 6673, 'CO/OL/04 SIERRA MORENA OCCIDENTAL': 6646, 'SE/OL/02 ESTEPA SUR': 6515, 'CO/OL/05 LAS COLONIAS-VEGA BAJA': 6346, 'CO/OL/03 SIERRA MORENA GUADIATO': 5

In [45]:
def preprocess_zonabio(data_frame):

  data_frame2=data_frame.copy()

  #Check for the parcelas that might have both null values and an secano/regadio
  dataframe_nonnull_perparcela=data_frame2.groupby('codparcela')["120_zona_biológica_raif"].first().dropna().to_dict()
  for index, row in data_frame2.iterrows():
    if row["codparcela"] in dataframe_nonnull_perparcela:
      data_frame2.at[index,"120_zona_biológica_raif"] = dataframe_nonnull_perparcela[row["codparcela"]]

  data_frame2["120_zona_biológica_raif"]=data_frame2["120_zona_biológica_raif"].cat.add_categories('SIN DEFINIR')
  """
  for index, row in data_frame2.iterrows():
    if row["120_zona_biológica_raif"] is np.nan:
      data_frame2.at[index, "120_zona_biológica_raif"] = "SIN DEFINIR"
  """

  data_frame2["120_zona_biológica_raif"] = data_frame2["120_zona_biológica_raif"].fillna("SIN DEFINIR")

  return data_frame2

Checks

In [46]:
feature_frame_preprocess_zonabio=preprocess_zonabio(feature_frame)
print(feature_frame_preprocess_zonabio["120_zona_biológica_raif"].isnull().sum())

0


In [47]:
counter_zonabio=Counter(feature_frame_preprocess_zonabio["120_zona_biológica_raif"])
print(counter_zonabio)

Counter({'GR/OL/07 IZNALLOZ': 48297, 'MA/OL/04 ANTEQUERA-NORTE': 46180, 'JA/OL/12 LOMA BAJA': 43433, 'GR/OL/01 MONTEFRIO': 41921, 'CO/OL/10 CAMPIÑA ALTA OCCIDENTAL': 27510, 'JA/OL/08 SIERRA AHILLOS Y SAN PEDRO': 24023, 'GR/OL/02 LA VEGA': 16523, 'CO/OL/08 CAMPIÑA ALTA ORIENTAL I': 14529, 'CO/OL/06 CAMPIÑA BAJA OCCIDENTAL': 13625, 'MA/OL/07 ANTEQUERA OCCIDENTAL': 12903, 'JA/OL/02 LOMA ALTA': 11975, 'CO/OL/02 SIERRA MORENA CENTRAL': 11470, 'CO/OL/09 CAMPIÑA ALTA ORIENTAL II': 11414, 'JA/OL/13 CAMPIÑA SUR': 10654, 'JA/OL/01 SIERRA MORENA': 10354, 'SE/OL/01 ESTEPA NORTE': 9452, 'JA/OL/05 SIERRA SUR': 8771, 'JA/OL/15 SIERRA CAZORLA': 8113, 'SE/OL/08 CAMPIÑA': 8096, 'GR/OL/03 ALHAMA DE GRANADA': 7759, 'JA/OL/14 CAMPIÑA NORTE': 7535, 'JA/OL/06 SIERRA SEGURA': 7384, 'CO/OL/11 SUBBÉTICA SEPTENTRIONAL': 7183, 'CA/OL/04 OLVERA': 7139, 'CO/OL/05 LAS COLONIAS-VEGA BAJA': 7092, 'SE/OL/02 ESTEPA SUR': 6315, 'CO/OL/04 SIERRA MORENA OCCIDENTAL': 6108, 'JA/OL/04 MAGINA NORTE': 6003, 'CO/OL/07 CAMPIÑA BA

59. 310_patrón_variedad_principal

We check the null values according to the landplot.

It is not entirely clear to what the patron variedad principal refers to:

- Picual
- Hojiblanca
- Manzanilla
- Arbequina
- Lechin
- Verdial
- Sin definir
Same philosophy as before:

1. Check if other parcelas have the same values and copy across them.
2. Homogeneize the values
3. For the remaining null values, add to not defined.

In [48]:
print(feature_frame["310_patrón_variedad_principal"].isnull().sum())
counter_patronvariedadprincipal=Counter(feature_frame["310_patrón_variedad_principal"])
print(counter_patronvariedadprincipal)

410071
Counter({nan: 410071, 'Picual': 46345, 'PICUAL': 34192, 'Hojiblanca': 13183, 'Ninguno': 10588, 'NINGUNO': 8170, 'Acebuche': 5999, 'Lechin de Sevilla': 2564, 'Manzanilla': 1312, 'HOJIBLANCA': 1094, 'ACEBUCHE': 1016, 'NEVADILLO': 553, 'Arbequina': 431, 'Verdial de Huevar': 180, 'SIN DEFINIR': 139, 'LECHIN': 138, 'NO': 111, 'hojiblanco': 72, 'HOHIBLANCA': 71, 'NINGUNA': 67, 'ARBEQUINA': 44, 'hojiblanca': 31, 'I 18': 27, 'AUTOENRAIZADA (NO INJERTADO)': 26, 'PICO LIMON': 19, 'PICUAL O MARTEÑA': 1})


In [49]:
def preprocess_patrvarprinc(data_frame):

  data_frame2=data_frame.copy()

  #Check for the parcelas that might have both null values and an secano/regadio
  dataframe_nonnull_perparcela=data_frame2.groupby('codparcela')["310_patrón_variedad_principal"].first().dropna().to_dict()
  for index, row in data_frame2.iterrows():
    if row["codparcela"] in dataframe_nonnull_perparcela:
      data_frame2.at[index,"310_patrón_variedad_principal"] = dataframe_nonnull_perparcela[row["codparcela"]]

  #Sorting out homogeneization.
  replacements_orientacion = {"310_patrón_variedad_principal": {"PICUAL O MARTEÑA": "PICUAL",
                                  "Picual": "PICUAL",
                                  "Hojiblanca": "HOJIBLANCA",
                                  "Ninguno": "SIN DEFINIR",
                                  "NINGUNO": "SIN DEFINIR",
                                  "Acebuche": "ACEBUCHE",
                                  "Lechin de Sevilla": "LECHIN",
                                  "Manzanilla": "MANZANILLA",
                                  "NEVADILLO": "PICUAL",
                                  "Arbequina": "ARBEQUINA",
                                  "Verdial de Huevar": "VERDIAL",
                                  "NO": "SIN DEFINIR",
                                  "hojiblanco": "HOJIBLANCA",
                                  "HOHIBLANCA": "HOJIBLANCA",
                                  "NINGUNA": "SIN DEFINIR",
                                  "hojiblanca": "HOJIBLANCA",
                                   "I 18": "SIN DEFINIR",
                                   "AUTOENRAIZADA (NO INJERTADO)": "SIN DEFINIR",
                                   "PICO LIMON": "SIN DEFINIR"
                                  }}
  data_frame2.replace(replacements_orientacion, inplace=True)
  """
  for index, row in data_frame2.iterrows():
    if row["310_patrón_variedad_principal"] is np.nan:
      data_frame2.at[index, "310_patrón_variedad_principal"] = "SIN DEFINIR"
  """

  data_frame2["310_patrón_variedad_principal"] = data_frame2["310_patrón_variedad_principal"].fillna("SIN DEFINIR")

  return data_frame2

Checks

In [50]:
feature_frame_preprocess_patrvarprinc=preprocess_patrvarprinc(feature_frame)
print(feature_frame_preprocess_patrvarprinc["310_patrón_variedad_principal"].isnull().sum())

0


In [51]:
counter_patrvarprinc=Counter(feature_frame_preprocess_patrvarprinc["310_patrón_variedad_principal"])
print(counter_patrvarprinc)

Counter({'SIN DEFINIR': 415919, 'PICUAL': 90247, 'HOJIBLANCA': 15215, 'ACEBUCHE': 9697, 'LECHIN': 3043, 'MANZANILLA': 1376, 'ARBEQUINA': 767, 'VERDIAL': 180})
